In [1]:
#Common tasks libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Miscellaeneous libraries.
import re
from datetime import timedelta
from joblib import dump, load

#Common tasks sklearn libraries.
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg


import warnings
warnings.filterwarnings('ignore')

In [2]:
def add_datepart(df, fldname, drop=True,errors="raise"):	
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.
    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_quarter_end', 'Is_quarter_start']
    for n in attr:
        if n == 'Week':
            df[targ_pre + n] = [ts.isocalendar().week for ts in fld]
        else:
            df[targ_pre + n] = getattr(fld.dt, n.lower())
    if drop: df.drop(fldname, axis=1, inplace=True)

In [3]:
#Vamos a suponer que day of week de asueto tiene que ser en 4 que es friday
def adjust_holidays(prevdf,daycol,boolcol):
    df = prevdf[prevdf[boolcol] == True]
    lst = []
    for i,dfiter in enumerate(df[daycol]):
        dayweek = dfiter.dayofweek
        dateNum = 4 - dayweek
        newDate = []
        newDate = [pd.Timestamp(dfiter.to_pydatetime() + timedelta(days=dateNum))]
        for item in df[[j for j in df.columns if j not in (daycol,boolcol)]].iloc[i].values:
            newDate.append(item)
        newDate.append(False)
        lst.append(newDate)
    newdf = prevdf[prevdf[boolcol] == False]
    newdf.reset_index(inplace = True, drop = True)
    newreg = pd.DataFrame(lst,columns = df.columns)
    return pd.concat(objs = (newdf,newreg))

holidays_df = pd.read_csv('./data/holidays_events.csv', parse_dates = ['date'])
mask_holidays = holidays_df['date'].dt.year != 2012
holidays_df_new = holidays_df[mask_holidays]
holidays_df_new.reset_index(inplace = True, drop = True)
holidays_df_new = adjust_holidays(holidays_df_new,'date','transferred')
holidays_df_new

,date,type,locale,locale_name,description,transferred
0,2013-01-01,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
2,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
3,2013-02-11,Holiday,National,Ecuador,Carnaval,False
4,2013-02-12,Holiday,National,Ecuador,Carnaval,False
...,...,...,...,...,...,...
6,2017-04-14,Holiday,Local,Cuenca,Fundacion de Cuenca,False
7,2017-05-26,Holiday,National,Ecuador,Batalla de Pichincha,False
8,2017-08-11,Holiday,National,Ecuador,Primer Grito de Independencia,False
9,2017-09-29,Holiday,Local,Ibarra,Fundacion de Ibarra,False


In [4]:
oil_prices_df = pd.read_csv('./data/oil.csv')
oil_prices_df = oil_prices_df.dropna()
print(oil_prices_df.shape)
add_datepart(oil_prices_df,'date',False)
oil_prices_df.head()

(1175, 2)


,date,dcoilwtico,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_quarter_end,Is_quarter_start
1,2013-01-02,93.14,2013,1,1,2,2,2,False,False
2,2013-01-03,92.97,2013,1,1,3,3,3,False,False
3,2013-01-04,93.12,2013,1,1,4,4,4,False,False
4,2013-01-07,93.20,2013,1,2,7,0,7,False,False
5,2013-01-08,93.21,2013,1,2,8,1,8,False,False


In [5]:
stores_df = pd.read_csv('./data/stores.csv')
print(stores_df.shape)
stores_df.head()

(54, 5)


,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [6]:
# Join with holidays
def holidays_on_dataset(df, dfhol, dfcols, dfholcols, dfcolsfinal):
    #values
    dfcols1, dfcols2, dfcols3 = dfcols
    dfholcols1, dfholcols2, dfholval = dfholcols
    
    #local holidays
    local_holidays = pd.merge(left = df, right = dfhol, how = 'inner', 
                             right_on = dfholcols1, left_on = dfcols1)
    local_holidays['is_holiday_local'] = 1
    dfcolsfinal.append('is_holiday_local')
    
    #national holidays
    national_holidays = pd.merge(left = df[[dfcols2]], right = dfhol, how = 'inner', on = dfcols2)
    national_holidays = national_holidays[national_holidays[dfholcols2] == dfholval].reset_index(drop = True)
    national_holidays['is_holiday_national'] = 1
    dfcolsfinal.append('is_holiday_national')
    
    #holidays
    local_national_holidays = pd.concat(objs = (local_holidays, national_holidays)).reset_index(drop = True)
    local_national_holidays = local_national_holidays[dfcols3]
    
    #Fill nulls
    lstvals = [dfholval,0,0]
    for elem,name in enumerate(local_national_holidays.columns[1:]):
        local_national_holidays[name] = local_national_holidays[name].fillna(lstvals[elem])
    
    #new holidays df
    local_national_holidays.drop_duplicates(inplace = True)
    local_national_holidays.sort_values(by = dfcols2, inplace = True)
    local_national_holidays.reset_index(inplace = True, drop = True)
    
    #Join to df
    new_df = pd.merge(left = df, right = local_national_holidays, how = 'left', on = dfcols2)
    
    #grab by columns
    new_df = new_df[dfcolsfinal]
    new_df['is_holiday_local'] = new_df['is_holiday_local'].fillna(0)
    new_df['is_holiday_national'] = new_df['is_holiday_national'].fillna(0)
    
    #Return the df now refactorized
    return new_df

In [7]:
def one_hot_encoding(df,col):
    onehot = OneHotEncoder()
    new_cols = onehot.fit_transform(df[[col]])
    one_hot_df = pd.DataFrame(new_cols.toarray(), columns = df[col].unique()) 
    df.drop(columns = col,inplace = True)
    return df.join(one_hot_df)      

def label_encoder(df,*colsname):
    label_encoding = LabelEncoder()
    for name in colsname:
        newName = "{}_encode".format(name)
        df[newName] = label_encoding.fit_transform(df[name])
        df.drop(columns = name, inplace = True)

In [8]:
test_df = pd.read_csv('./data/test.csv', parse_dates = ['date'], index_col = 0)
test_df['index_original'] = test_df.index
test_group_df = test_df.groupby(by=['family','date','store_nbr']).sum().reset_index()

#join with stores
test_join_df = pd.merge(left = test_group_df, right = stores_df, on = 'store_nbr', how = 'inner')
add_datepart(test_join_df,'date',False)

#Adjust holidays
test_join_df = holidays_on_dataset(test_join_df, holidays_df_new, 
                [['date','city'],'date',['date','city','is_holiday_local','is_holiday_national']],
                [['date','locale_name'],'locale','National'],
                ['family','date','store_nbr','onpromotion','city_x','state','type','cluster',
                'Year','Month','Week','Day','Dayofweek','Dayofyear','Is_quarter_end','Is_quarter_start','index_original'])


test_join_df_encode = test_join_df.copy()
label_encoder(test_join_df_encode, 'Is_quarter_end','Is_quarter_start','city_x','state','type')
test_join_df_encode = pd.merge(left = test_join_df_encode, right=oil_prices_df[['date','dcoilwtico']], on='date', how='left')
test_join_df_encode['oil_price'] = test_join_df_encode['dcoilwtico'].shift().rolling(7, min_periods=1).mean().fillna(test_join_df_encode['dcoilwtico'])
test_join_df_encode.drop(['dcoilwtico'], axis=1, inplace=True)
test_join_df_encode.at[0, 'oil_price'] = 93.14
test_join_df_encode = one_hot_encoding(test_join_df_encode,'family')

In [9]:
test_join_df_encode.to_csv('./data/test_join_df_encoded_one_hot.csv')

In [10]:
test_dataset_clean = pd.read_csv('./data/test_join_df_encoded_one_hot.csv')
test_dataset_clean.drop('date', axis=1, inplace=True)
test_dataset_clean.drop('Unnamed: 0', axis=1, inplace=True)

In [11]:
test_dataset_clean.sort_values(by = 'index_original', inplace = True)

In [12]:
test_dataset_clean

,store_nbr,onpromotion,cluster,Year,Month,Week,Day,Dayofweek,Dayofyear,index_original,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,1,0,13,2017,8,33,16,2,228,3000888,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,1,0,13,2017,8,33,16,2,228,3000889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,1,2,13,2017,8,33,16,2,228,3000890,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,1,20,13,2017,8,33,16,2,228,3000891,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,1,0,13,2017,8,33,16,2,228,3000892,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4687,9,1,6,2017,8,35,31,3,243,3029395,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4703,9,0,6,2017,8,35,31,3,243,3029396,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4719,9,1,6,2017,8,35,31,3,243,3029397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4735,9,9,6,2017,8,35,31,3,243,3029398,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
#Modelo 1
clean_dataset = pd.read_csv('./data/test_join_df_encoded.csv')
br = load('BRnormalmode.joblib')
submit = pd.read_csv('./data/baggingregressor_submission.csv')
submit

,id,sales
0,3000888,4.2
1,3000889,5.1
2,3000890,6.9
3,3000891,6.5
4,3000892,0.7
...,...,...
28507,3029395,1.5
28508,3029396,4.8
28509,3029397,5.1
28510,3029398,2.2


In [14]:
#Modelo 2
sk = pd.read_csv('skforecasting_rf.csv')
sk

,id,sales
0,3000888,2.87
1,3000889,0.22
2,3000890,1.43
3,3000891,15.64
4,3000892,0.22
...,...,...
28507,3029395,15.59
28508,3029396,15.59
28509,3029397,15.59
28510,3029398,6.65


In [15]:
#Modelo 3
rf = load('RFfinalmode.joblib')
test_values = rf.predict(test_dataset_clean)
submissions = pd.read_csv('./data/sample_submission.csv',index_col = 'id')
submissions['sales'] = test_values
submissions.to_csv('espero_que_salga_bien.csv')
submissions

,sales
id,
3000888,3.200000
3000889,0.000000
3000890,6.300000
3000891,2411.300000
3000892,0.000000
...,...
3029395,309.806004
3029396,91.831600
3029397,1156.511100
